# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Queue Management

**Subtopic** - Implementing a work-stealing queue scenario with multiple queues (advanced concurrency scenario)

**Use Case** - Implement a Python module that simulates a work-stealing scenario with multiple queues. The module should manage tasks distributed across multiple worker threads. Each worker thread should have its own queue of tasks. If a worker thread exhausts its queue, it should be able to steal tasks from the queue of another worker. The module should also provide functions to add tasks to the queues, to start the worker threads, and to query the state of the queues and workers.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt
Problem Statement:
- Implement a Python module that simulates a work-stealing scenario with multiple queues.
- The module should manage tasks distributed across multiple worker threads.
- Each worker thread should have its own queue of tasks. Worker threads are a simulation.
- If a worker thread exhausts its queue, it should be able to steal tasks from the queue of another worker.
- The module should also provide functions to add tasks to the queues, to start the worker threads, and to query the state of the queues and workers.

Input Format:
- INIT <num_workers> : Initializes the simulator with the specified number of workers.
- ADD <worker_id> <task> : Adds a task (string) to the queue of the specified worker.
- START <worker_id> : Starts the specified worker processing one task. If its queue is empty, it automatically steals a task from another worker.
- STATUS : Prints the current state of all worker queues.
- QUIT : Terminates the simulation.

Input Constraints:
- 1 <= num_workers <= 100
- 0 <= worker_id < num_workers
- Task names are non-empty strings with no spaces, up to 100 characters.
- Total number of commands will not exceed 10000.
- At most 1000 tasks can exist across all queues at any time.
- START will only be called after INIT.
- STATUS and QUIT can be called any time after INIT.
- All worker IDs used in commands are guaranteed to be valid.
- INIT may be called only once at the beginning.

Output Format:
- For each STATUS command, output one line per worker in the format:  Worker <worker_id>: [<task1>, <task2>, ..., <taskN>]
- The order of tasks is from front to back of the worker's queue.
- If a worker's queue is empty, print: Worker <worker_id>: []
- No output is produced for INIT, ADD, START, or QUIT commands.

Class Definition:
- Class Name: WorkStealingSimulator

Function Signature:
```python
def process_commands(commands: list[str]) -> list[str]
```

# Requirements
Explicit Requirements:
- Each worker must have its own task queue.
- A worker must automatically steal a task when its queue is empty during START.
- The simulator must support INIT, ADD, START, STATUS, and QUIT commands.
- Commands are processed one at a time, in the order provided.
- When a worker processes a task (either from its own queue or by stealing), that task is removed from the queue.

Implicit Requirements:
- When stealing, the worker will scan other workers in increasing order of worker_id and steal the first available task.
- START must be a no-op if no tasks are available to run or steal.
- STATUS must always reflect the up-to-date state after all previous operations.
- Task names may include alphanumeric characters and symbols except whitespace.

Solution Expectations:
- The solution must define a class named WorkStealingSimulator.
- The class must process a list of input commands and return a list of output strings for STATUS commands.
- Task execution and automatic stealing must be correctly simulated for each START command.
- The implementation must ensure valid input parsing and maintain correct internal state.
- No output should be produced for INIT, ADD, START, or QUIT commands.
- The output must strictly follow the format: "Worker <id>: [<task1>, <task2>, ...]" for each worker in STATUS.
- The solution must handle up to 100 workers and 10,000 commands efficiently.

Edge Cases and Behaviour:
- START on a worker when all queues are empty: No task is executed; system remains unchanged.
- START on a worker with an empty queue: The worker automatically steals the first available task from the lowest-indexed non-empty worker.
- STATUS when no tasks have been added: All workers show empty queues.
- Multiple START commands in a row: Each START should process one task per call, either from own queue or by stealing.
- ADD to a non-zero worker: Task must be queued only to the specified worker and not processed until START is called.
- INIT with maximum workers (100): The system must still correctly manage all queues and operations without performance issues.
- QUIT immediately after INIT: Should end the simulation with no output.
- STATUS after all tasks have been consumed: All workers should display empty queues.

In [ ]:
# code
"""
Simulates a multi-worker task scheduler with work stealing logic.

Processes commands like INIT, ADD, START, STATUS, and QUIT to manage
and track worker queues for task distribution and execution.
"""


class WorkStealingSimulator:
    """Simulates task distribution and work stealing among multiple workers."""

    def __init__(self):
        """Initialize the simulator with no workers and default flags."""
        self.num_workers = 0
        self.workers = []
        self._initialized = False  # Flag to ensure INIT is called only once
        self._terminated = False   # Flag to stop processing after QUIT

    def _init_simulator(self, num_workers: int):
        """Initialize worker queues."""
        if self._initialized:
            return
        if not (1 <= num_workers <= 100):
            return
        self.num_workers = num_workers
        self.workers = [[] for _ in range(num_workers)]
        self._initialized = True

    def _add_task(self, worker_id: int, task: str):
        """Add a task to a specific worker's queue if valid."""
        if not self._initialized:
            return
        if not task or ' ' in task or len(task) > 100:
            return
        if 0 <= worker_id < self.num_workers:
            self.workers[worker_id].append(task)

    def _start_worker(self, worker_id: int):
        """Start task execution from the worker or steals a task if idle."""
        if not self._initialized:
            return
        if 0 <= worker_id < self.num_workers:
            if self.workers[worker_id]:
                self.workers[worker_id].pop(0)
            else:
                for i in range(self.num_workers):
                    if i == worker_id:
                        continue
                    if self.workers[i]:
                        self.workers[i].pop(0)
                        return

    def _get_status(self) -> list[str]:
        """Return the current task queue status for all workers."""
        if not self._initialized:
            return []
        status_output = []
        for i, queue in enumerate(self.workers):
            tasks_str = ", ".join(queue)
            status_output.append(f"Worker {i}: [{tasks_str}]")
        return status_output

    def process_commands(self, commands: list[str]) -> list[str]:
        """Process a list of simulation commands."""
        output = []

        for command in commands:
            if self._terminated:
                break

            parts = command.split()
            if not parts:
                continue

            cmd_type = parts[0]

            try:
                if cmd_type == "INIT":
                    if len(parts) == 2:
                        num_workers = int(parts[1])
                        self._init_simulator(num_workers)
                elif cmd_type == "ADD":
                    if len(parts) == 3:
                        worker_id = int(parts[1])
                        task = parts[2]
                        self._add_task(worker_id, task)
                elif cmd_type == "START":
                    if len(parts) == 2:
                        worker_id = int(parts[1])
                        self._start_worker(worker_id)
                elif cmd_type == "STATUS":
                    if len(parts) == 1:
                        output.extend(self._get_status())
                elif cmd_type == "QUIT":
                    if len(parts) == 1:
                        self._terminated = True
                        break
                else:
                    pass  # Unknown command
            except (ValueError, IndexError):
                pass  # Invalid or incomplete command

        return output


In [ ]:
# tests

"""Unit tests for WorkStealingSimulator."""

import unittest
from main import WorkStealingSimulator


class TestWorkStealingSimulator(unittest.TestCase):
    """Test behaviors of WorkStealingSimulator."""

    def test_init_and_status_empty(self):
        """Initialize workers and check empty status."""
        sim = WorkStealingSimulator()
        output = sim.process_commands(["INIT 3", "STATUS"])
        self.assertEqual(output, [
            "Worker 0: []",
            "Worker 1: []",
            "Worker 2: []"
        ])

    def test_add_and_status(self):
        """Add tasks to workers and verify status."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 2", "ADD 0 taskA", "ADD 1 taskB", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, [
            "Worker 0: [taskA]",
            "Worker 1: [taskB]"
        ])

    def test_start_own_queue(self):
        """Start task on same worker's own queue."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 2", "ADD 0 t1", "START 0", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, [
            "Worker 0: []",
            "Worker 1: []"
        ])

    def test_start_steal(self):
        """Start task by stealing from another worker."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 2", "ADD 1 t2", "START 0", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, [
            "Worker 0: []",
            "Worker 1: []"
        ])

    def test_start_no_tasks(self):
        """Attempt to start when no tasks exist."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 2", "START 0", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, [
            "Worker 0: []",
            "Worker 1: []"
        ])

    def test_multiple_starts_and_steals(self):
        """Test repeated starts across multiple workers."""
        sim = WorkStealingSimulator()
        cmds = [
            "INIT 3", "ADD 0 t1", "ADD 1 t2", "ADD 2 t3",
            "START 0", "START 0", "START 0", "STATUS"
        ]
        output = sim.process_commands(cmds)
        self.assertEqual(output, [
            "Worker 0: []",
            "Worker 1: []",
            "Worker 2: []"
        ])

    def test_status_after_all_consumed(self):
        """Verify status after all tasks are started."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 2", "ADD 0 t1", "START 0", "START 0", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, ["Worker 0: []", "Worker 1: []"])

    def test_add_to_nonzero_worker(self):
        """Add task to a non-zero indexed worker."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 3", "ADD 2 tX", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, [
            "Worker 0: []",
            "Worker 1: []",
            "Worker 2: [tX]"
        ])

    def test_quit_immediately(self):
        """Test quit before other commands."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 2", "QUIT", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, [])

    def test_status_after_quit(self):
        """Check that no status appears after quit."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 2", "ADD 0 t1", "QUIT", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, [])

    def test_max_workers(self):
        """Add tasks to 100 workers and check status."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 100"] + [f"ADD {i} t{i}" for i in range(100)] + ["STATUS"]
        output = sim.process_commands(cmds)
        for i in range(100):
            self.assertEqual(output[i], f"Worker {i}: [t{i}]")

    def test_multiple_status(self):
        """Check correct state across multiple status calls."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 2", "ADD 0 t1", "STATUS", "START 0", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, [
            "Worker 0: [t1]",
            "Worker 1: []",
            "Worker 0: []",
            "Worker 1: []"
        ])

    def test_task_name_constraints(self):
        """Ensure invalid task names are ignored."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 2", "ADD 0 invalid task", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, ["Worker 0: []", "Worker 1: []"])

    def test_start_on_all_empty(self):
        """Start on each worker when all queues are empty."""
        sim = WorkStealingSimulator()
        cmds = ["INIT 3", "START 0", "START 1", "START 2", "STATUS"]
        output = sim.process_commands(cmds)
        self.assertEqual(output, [
            "Worker 0: []",
            "Worker 1: []",
            "Worker 2: []"
        ])


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/91ddd4e2-c0da-4e7a-b34e-676f84c912d7?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F75166%2Fview

#### Model code:

```python
# code generated by the model
class WorkStealingSimulator:
    def __init__(self):
        self.num_workers = 0
        self.queues = []
        self.initialized = False

    def init(self, num_workers: int):
        self.num_workers = num_workers
        self.queues = [[] for _ in range(num_workers)]
        self.initialized = True

    def add_task(self, worker_id: int, task: str):
        self.queues[worker_id].append(task)

    def start_worker(self, worker_id: int):
        if not self.initialized:
            return
        
        if self.queues[worker_id]:
            # This worker processes a task if its queue is not empty
            self.queues[worker_id].pop(0)
        else:
            # Attempt to "steal" a task from other workers
            for i in range(self.num_workers):
                if i != worker_id and self.queues[i]:
                    self.queues[i].pop(0)
                    break

    def status(self) -> list[str]:
        statuses = []
        for i, queue in enumerate(self.queues):
            tasks = ', '.join(queue)
            statuses.append(f"Worker {i}: [{tasks}]")
        return statuses

    def process_commands(self, commands: list[str]) -> list[str]:
        output = []
        for command in commands:
            parts = command.split()
            action = parts[0]
            # <Issue>: Allows INIT to be called multiple times without restriction.
            # <Issue>: Invalid commands are not handled gracefully
            # <Issue>: Does not validate Worker ID bounds, task name formats, command format.
            if action == "INIT":
                num_workers = int(parts[1])
                self.init(num_workers)
            elif action == "ADD":
                worker_id = int(parts[1])
                task = parts[2]
                self.add_task(worker_id, task)
            elif action == "START":
                worker_id = int(parts[1])
                self.start_worker(worker_id)
            elif action == "STATUS":
                statuses = self.status()
                output.extend(statuses)
            elif action == "QUIT":
                break
        return output

# Example of how to use the class:
# simulator = WorkStealingSimulator()
# output = simulator.process_commands([
#     "INIT 3",
#     "ADD 0 Task1",
#     "ADD 1 Task2",
#     "START 0",
#     "START 1",
#     "STATUS",
#     "QUIT"
# ])
# for line in output:
#     print(line)
```